1- Copy all the images from the workbook into a new directory using shutil and pathlib
2- Create a new workbook
3- Extract text from images
4- Extract text from excel 
5- Store the extracted text in a new excel workbook
6- Use NLP to make the data consistent

In [ ]:
# Copy all the images into a new folder

In [8]:
import shutil # Used to copy the images to new path and to remove the temp directory
import pathlib # Helps in making the paths consistent and helps to join different directories or folders for transferring data
import zipfile # Works when excel files are zipped
import os # Helps to interact with the operating system

In [9]:
def extract_images_from_excel(path, output_folder_name='extracted_images'):
    """
    Extracts images from an Excel file and stores them in a single folder.

    Args:
        path (pathlib.Path or str): Excel file path.
        output_folder_name (str): Name of the folder to store the extracted images.
            Defaults to 'extracted_images'.

    Returns:
        new_paths (list[pathlib.Path]): List of paths to the extracted images.
    """
    # Convert path to pathlib.Path if it's a string
    if isinstance(path, str):
        path = pathlib.Path(path)

    # Check if the file has the '.xlsx' extension
    if path.suffix != '.xlsx':
        raise ValueError('Path must be an xlsx file')

    # Extract the filename (excluding the extension) using .stem
    name = path.stem

    # Create a new folder for the extracted images
    output_folder = path.parent / output_folder_name
    output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

    # Create a temporary directory for unzipping the Excel file
    temp_dir = path.parent / 'temp'
    temp_dir.mkdir(exist_ok=True)  # Create folder if it doesn't exist

    try:
        # Unzip the Excel file into the temporary directory
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Locate the 'media' directory within the unzipped content
        media_dir = temp_dir / 'xl' / 'media'

        image_index = 0  # Initialize an index for the images
        new_paths = []  # List to store the paths of the extracted images

        # Iterate through the files in the 'media' directory
        for root, dirs, files in os.walk(media_dir):
            for file in files:
                image_index += 1  # Increment the image index for each image found

                # Construct paths for the original image and the new destination
                image_path = pathlib.Path(root) / file
                new_path = output_folder / f'{name}-{str(image_index)}.png'

                # Copy the image to the output folder with a new name
                shutil.copy(image_path, new_path)

                # Store the new path in the list
                new_paths.append(new_path)

    finally:
        # Cleanup: Remove the temporary directory
        shutil.rmtree(temp_dir)

    # Return the list of paths to the extracted images
    return new_paths


In [10]:
excel_file_path = "C:/Users/Shreshtha/Downloads/Project UHC/Trial 2.xlsx"
extracted_image_paths = extract_images_from_excel(excel_file_path)

In [11]:
# Path to the directory containing the images
images_directory = "C:/Users/Shreshtha/Downloads/Project UHC/extracted_images"

In [ ]:
# Extracting text from excel and images

In [12]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os
import pandas as pd
import cv2
from PIL import Image
import pytesseract

In [13]:
# Create a new Excel workbook
new_workbook = Workbook()


In [14]:
# Function to extract text from an image using OCR

In [15]:
# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [16]:
# Dictionary to store text data with filenames as keys
text_data_dict = {}

In [17]:
# Function to extract regular text from an image using OCR
def extract_text_from_image(image_path):
    try:
        text = pytesseract.image_to_string(Image.open(image_path))
        return text.strip()
    except Exception as e:
        print(f"Error extracting text from image {image_path}: {e}")
        return None

In [19]:
# Function to extract data from an Excel sheet
def extract_data_from_excel(excel_file_path, sheet_name):
    workbook = load_workbook(excel_file_path, read_only=True, data_only=True)
    sheet_df = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=None)
    return sheet_df

In [20]:
# Load the existing Excel data
excel_data_dict = {}

In [ ]:
# Adding the extracted excel data in a new workbook

In [26]:
# Iterate through each sheet in the original workbook
for sheet_name in load_workbook(excel_file_path, read_only=True).sheetnames:
    # Extract data from the Excel sheet
    excel_data_df = extract_data_from_excel(excel_file_path, sheet_name)
    
    # Print the Excel data
    print(f"Data from {sheet_name}:\n{excel_data_df}\n")

    # Create a new sheet in the workbook for each Excel sheet
    excel_sheet = new_workbook.create_sheet(title=f"Excel_{sheet_name}")

    # Write the data to the sheet
    for row in dataframe_to_rows(excel_data_df, index=False, header=True):
        excel_sheet.append(row)

    # Store the DataFrame in the dictionary
    excel_data_dict[sheet_name] = excel_data_df

Data from Sheet1:
                                                    0  \
0                                         Group Name:   
1                            2 Group Number (or TBD):   
2                                     3 Request Date:   
3                               Effective Date of NSB   
4   Please Note: All tersisted on this form are su...   
5                                                 NaN   
6                                                 NaN   
7                             $ General exformatiqued   
8                                      CSP (Optional)   
9   Base (Similar standard Plan) Tracking ID# (REQ...   
10                            # of Employees Enrolled   
11                           Group State (NY, NJ, CT)   
12                      Market (Large or Small Group)   
13  Product (eg HMO, POS, Classic Access, Drect, O...   
14                        Access (Gated or Non-Gated)   
15          Network Freedom, Liberty, or Choice Plus)   
16           

In [ ]:
# Adding the extracted image data to the new workbook

In [24]:
# Iterate through each image in the directory
for filename in os.listdir(images_directory):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(images_directory, filename)

        # Extract regular text
        text_data_regular = extract_text_from_image(image_path)

        # Print the image data
        print(f"Data from Image {filename}:\n{text_data_regular}\n")

        text_data_dict[filename] = text_data_regular

# Create a new sheet for each image's text data
for filename, text_data in text_data_dict.items():
    text_sheet = new_workbook.create_sheet(title=f"Image_{filename}")
    text_sheet['A1'] = text_data

Data from Image Trial 2-1.png:
Group Heme: Aurora Products
Group Number (or TBD): 1264561
Request Date: 7214/2020
Effective Date of NSB: 31/2020

Please Note: All tems isied on this form are subject to review, Please highlight the NSB's in yellow,

Benefits Currently in Place Requested Benefits

Current Plan 4: Current Plan 2:
0100001 BUY UP 0100002 CORE

Base (Similar standard Plan) Tracking ID # (REQUIRED)
+ Reach out to your Underwriter

+ 4 14
# of Employees Enrolled: 39 2 25 % 26 15
Group State (NY, NJ, CT) cr cr cr. cr cr cr
Market (Large or Small Group) Large Large Large Large Large Large
Product (e.9: HHO, POS, Classic, Access, Direct, OxUSA, Value Option
or EPO) HMO Huo HSA Huo HMO HSA
Access (Gated or Non-Gated) Non-Gated Gated Non-Gated Non-Gated Gates Non-Gated
Network (Freedom, Liberty, or Choice Pius) Freedom Freedom Freedom Freedom Freedom Freedom
Current Carrier (Oxford or Competitor) Polaris Polaris Polaris Polaris

Nore INE
PCPISpecialist OV Copay 30/50 30/50 D8 100%


In [23]:
# Save the new workbook
new_workbook.save("C:/Users/Shreshtha/Downloads/Project UHC/Combined Trial2.xlsx")

Problem- still difficult to standardise the extracted text

Solution- Categorise the excel sheets according to the type of image or data present